# QC-Py-05 - Universe Selection dans QuantConnect

> **Selection dynamique d'actifs pour des strategies multi-assets**
> Duree: 75 minutes | Niveau: Intermediaire | Python + QuantConnect

---

## Objectifs d'Apprentissage

A la fin de ce notebook, vous serez capable de :

1. Comprendre le concept d'**Universe** et son importance en trading algorithmique
2. Creer un **Manual Universe** avec une liste fixe de symboles
3. Implementer un **Coarse Universe Selection** base sur le volume et le prix
4. Utiliser le **Fine Universe Selection** avec des donnees fondamentales
5. Configurer les **Universe Settings** (resolution, leverage, etc.)
6. Gerer le **Dynamic Rebalancing** lors des changements d'univers
7. Planifier des rebalancements avec **Scheduled Universe Selection**
8. Construire une **strategie Momentum** complete

## Prerequis

- Notebooks QC-Py-01 a QC-Py-04 completes
- Comprehension du lifecycle QCAlgorithm
- Notions de base en analyse fondamentale

## Structure du Notebook

1. Introduction a l'Universe Selection (10 min)
2. Manual Universe (15 min)
3. Coarse Universe Selection (20 min)
4. Fine Universe Selection (20 min)
5. Universe Settings (10 min)
6. Dynamic Rebalancing (15 min)
7. Scheduled Universe Selection (10 min)
8. Strategie Complete: Momentum Universe (20 min)

---

## 1. Introduction a l'Universe Selection (10 min)

### Qu'est-ce qu'un Universe?

Dans QuantConnect, un **Universe** represente l'ensemble des actifs que votre algorithme considere pour le trading a un moment donne. Plutot que de trader toujours les memes actions, l'Universe Selection permet de :

- **Selectionner dynamiquement** les meilleurs candidats selon vos criteres
- **S'adapter aux conditions de marche** (nouvelle IPO, delisting, etc.)
- **Filtrer** parmi des milliers d'actifs pour ne garder que les plus pertinents
- **Automatiser** le screening qui prendrait des heures manuellement

### Pourquoi selectionner dynamiquement des actions?

| Approche | Avantages | Inconvenients |
|----------|-----------|---------------|
| **Liste fixe** | Simple, rapide | Risque de biais, actions obsoletes |
| **Universe dynamique** | Adaptable, sans biais | Plus complexe, backtest plus long |

**Exemples d'utilisation** :

1. **Strategie momentum** : Trader les 50 actions avec le meilleur momentum sur 12 mois
2. **Value investing** : Filtrer les actions avec P/E < 15 et dividende > 3%
3. **Secteur specifique** : Ne trader que les actions du secteur technologie
4. **Liquidite** : Exclure les actions avec un volume trop faible

### Types d'Universe Selection

```
Universe Selection
        |
        +-- Manual Universe (liste fixe)
        |
        +-- Coarse Universe (filtre sur prix, volume)
        |         |
        |         +-- Fine Universe (donnees fondamentales)
        |
        +-- Custom Universe (donnees alternatives)
```

### Pipeline de Selection

```
8000+ actions US          Coarse Selection          Fine Selection          Portfolio
    [Marche]      --->    (prix, volume)     --->  (P/E, secteur)    --->   [Trading]
                          ~500 candidats            ~50 candidats           ~20 actions
```

---

## 2. Manual Universe (15 min)

### Approche la plus simple : Liste fixe de symboles

La maniere la plus simple de definir un univers est d'ajouter manuellement les actions que vous souhaitez trader avec `self.AddEquity()`.

### Exemple : Portfolio FAANG

Les FAANG (META ex-Facebook, Apple, Amazon, Netflix, Google) sont les geants de la tech americaine. Creons un algorithme qui trade ces 5 actions.

In [ ]:
# Manual Universe : Portfolio FAANG
# A copier dans l'IDE QuantConnect

from AlgorithmImports import *

class FAANGManualUniverse(QCAlgorithm):
    """
    Exemple de Manual Universe avec les actions FAANG.
    Allocation equal-weight sur les 5 actions.
    """
    
    def Initialize(self):
        # Configuration backtest
        self.SetStartDate(2020, 1, 1)
        self.SetEndDate(2023, 12, 31)
        self.SetCash(100000)
        
        # Liste des symboles FAANG
        self.tickers = ["META", "AAPL", "AMZN", "NFLX", "GOOGL"]
        self.symbols = []
        
        # Ajouter chaque action manuellement
        for ticker in self.tickers:
            equity = self.AddEquity(ticker, Resolution.Daily)
            self.symbols.append(equity.Symbol)
            self.Log(f"Added {ticker}: {equity.Symbol}")
        
        # Allocation cible : equal-weight (20% chacune)
        self.target_allocation = 1.0 / len(self.symbols)
        
        # Rebalancement initial planifie
        self.first_trade = True
        
        self.Log(f"FAANG Universe initialized with {len(self.symbols)} symbols")
    
    def OnData(self, data):
        # Attendre que toutes les donnees soient disponibles
        if not all(data.ContainsKey(s) for s in self.symbols):
            return
        
        # Allocation initiale (une seule fois)
        if self.first_trade:
            for symbol in self.symbols:
                self.SetHoldings(symbol, self.target_allocation)
                self.Log(f"Initial allocation: {symbol.Value} -> {self.target_allocation:.1%}")
            self.first_trade = False
    
    def OnEndOfAlgorithm(self):
        self.Log(f"Final Portfolio Value: ${self.Portfolio.TotalPortfolioValue:,.2f}")

### Avantages et limites du Manual Universe

| Avantages | Limites |
|-----------|--------|
| Simple a implementer | Biais de selection (survivorship bias) |
| Controle total sur les actifs | Pas d'adaptation aux conditions de marche |
| Backtest rapide | Ne decouvre pas de nouvelles opportunites |
| Ideal pour tester une idee | Maintenance manuelle requise |

### Quand utiliser Manual Universe?

- **Prototypage rapide** : Tester une strategie sur quelques actions
- **Strategie sectorielle** : Trader un panier defini (ETF-like)
- **Arbitrage specifique** : Pairs trading sur 2-3 actifs
- **Benchmark** : Comparer avec une strategie dynamique

### Variation : Multiple Asset Classes

In [ ]:
# Manual Universe multi-asset
# Exemple avec actions, ETFs et crypto

from AlgorithmImports import *

class MultiAssetManualUniverse(QCAlgorithm):
    
    def Initialize(self):
        self.SetStartDate(2020, 1, 1)
        self.SetEndDate(2023, 12, 31)
        self.SetCash(100000)
        
        # Actions individuelles
        self.stocks = {
            "AAPL": self.AddEquity("AAPL", Resolution.Daily).Symbol,
            "MSFT": self.AddEquity("MSFT", Resolution.Daily).Symbol,
        }
        
        # ETFs
        self.etfs = {
            "SPY": self.AddEquity("SPY", Resolution.Daily).Symbol,   # S&P 500
            "QQQ": self.AddEquity("QQQ", Resolution.Daily).Symbol,   # Nasdaq 100
            "TLT": self.AddEquity("TLT", Resolution.Daily).Symbol,   # Obligations
        }
        
        # Tous les symboles
        self.all_symbols = list(self.stocks.values()) + list(self.etfs.values())
        
        self.Log(f"Multi-asset universe: {len(self.stocks)} stocks, {len(self.etfs)} ETFs")

---

## 3. Coarse Universe Selection (20 min)

### Filtrage base sur le prix et le volume

Le **Coarse Universe Selection** est le premier niveau de filtrage dynamique. Il permet de filtrer parmi toutes les actions du marche en utilisant des criteres simples :

- **Price** : Prix de l'action (eviter les penny stocks)
- **DollarVolume** : Volume en dollars (liquidite)
- **HasFundamentalData** : Presence de donnees fondamentales

### Structure du Coarse Selection

```python
def Initialize(self):
    # Activer la selection d'univers
    self.AddUniverse(self.CoarseSelectionFunction)

def CoarseSelectionFunction(self, coarse):
    # coarse = liste de CoarseFundamental objects
    # Retourne une liste de Symbol
    filtered = [x for x in coarse if x.Price > 10]
    return [x.Symbol for x in filtered]
```

### Proprietes de CoarseFundamental

| Propriete | Type | Description |
|-----------|------|-------------|
| `Symbol` | Symbol | Identifiant unique de l'action |
| `Price` | float | Prix de cloture |
| `DollarVolume` | float | Volume quotidien en dollars (Price x Volume) |
| `Volume` | long | Nombre d'actions echangees |
| `HasFundamentalData` | bool | True si donnees fondamentales disponibles |

In [ ]:
# Coarse Universe Selection : Top 100 par volume
# A copier dans l'IDE QuantConnect

from AlgorithmImports import *

class CoarseUniverseAlgorithm(QCAlgorithm):
    """
    Selection dynamique des 100 actions les plus liquides.
    Filtres : prix > 10$, volume > 10M$/jour.
    """
    
    def Initialize(self):
        self.SetStartDate(2020, 1, 1)
        self.SetEndDate(2023, 12, 31)
        self.SetCash(100000)
        
        # Configurer l'univers avec Coarse Selection
        self.AddUniverse(self.CoarseSelectionFunction)
        
        # Nombre d'actions a selectionner
        self.num_stocks = 100
        
        # Filtres
        self.min_price = 10           # Prix minimum ($)
        self.min_dollar_volume = 10000000  # Volume min ($10M/jour)
        
        self.Log(f"Coarse Universe initialized: top {self.num_stocks} by volume")
    
    def CoarseSelectionFunction(self, coarse):
        """
        Fonction de filtrage Coarse.
        Appelee automatiquement chaque jour par QuantConnect.
        
        Args:
            coarse: Liste de CoarseFundamental objects (toutes les actions)
        
        Returns:
            Liste de Symbol des actions selectionnees
        """
        # Etape 1: Filtrer par prix et volume
        filtered = [x for x in coarse 
                    if x.Price > self.min_price 
                    and x.DollarVolume > self.min_dollar_volume]
        
        # Etape 2: Trier par DollarVolume (decroissant)
        sorted_by_volume = sorted(filtered, 
                                   key=lambda x: x.DollarVolume, 
                                   reverse=True)
        
        # Etape 3: Garder les top N
        top_stocks = sorted_by_volume[:self.num_stocks]
        
        # Log pour debug
        self.Log(f"Coarse selection: {len(filtered)} filtered, {len(top_stocks)} selected")
        
        # Retourner les symboles
        return [x.Symbol for x in top_stocks]
    
    def OnData(self, data):
        # L'univers change automatiquement
        # self.ActiveSecurities contient les actions actuelles
        pass
    
    def OnSecuritiesChanged(self, changes):
        """Appele quand l'univers change (ajout/retrait d'actions)."""
        self.Log(f"Securities changed: +{len(changes.AddedSecurities)} / -{len(changes.RemovedSecurities)}")

### Comprendre le flux de Coarse Selection

```
Chaque jour de trading:

1. QuantConnect appelle CoarseSelectionFunction(coarse)
   |
   +-- coarse = 8000+ CoarseFundamental objects
   |
   +-- Votre logique de filtrage
   |
   +-- Retourne liste de Symbol (ex: 100 symboles)

2. QuantConnect compare avec l'univers precedent
   |
   +-- Nouvelles actions -> AddedSecurities
   +-- Actions retirees -> RemovedSecurities

3. OnSecuritiesChanged() est appele
   |
   +-- Vous pouvez ajuster vos positions

4. OnData() recoit les donnees des actions de l'univers actuel
```

### Optimisations importantes

**1. Utiliser HasFundamentalData pour eviter les ETFs et ADRs** :

```python
filtered = [x for x in coarse 
            if x.HasFundamentalData  # Exclut ETFs, ADRs
            and x.Price > 10 
            and x.DollarVolume > 10000000]
```

**2. Limiter la frequence de reselection** :

```python
def Initialize(self):
    # Reselection mensuelle au lieu de quotidienne
    self.UniverseSettings.Resolution = Resolution.Daily
    self.AddUniverse(self.CoarseSelectionFunction)
    self.last_month = -1

def CoarseSelectionFunction(self, coarse):
    # Ne re-selectionner qu'en debut de mois
    if self.Time.month == self.last_month:
        return Universe.Unchanged
    self.last_month = self.Time.month
    
    # Logique de filtrage...
```

In [ ]:
# Coarse Selection avec reselection mensuelle
# Optimisation pour reduire le temps de backtest

from AlgorithmImports import *

class MonthlyCoarseUniverse(QCAlgorithm):
    """
    Coarse selection avec rebalancement mensuel.
    Evite de recalculer l'univers chaque jour.
    """
    
    def Initialize(self):
        self.SetStartDate(2020, 1, 1)
        self.SetEndDate(2023, 12, 31)
        self.SetCash(100000)
        
        self.AddUniverse(self.CoarseSelectionFunction)
        
        self.num_stocks = 50
        self.last_month = -1  # Pour tracker le mois
    
    def CoarseSelectionFunction(self, coarse):
        # Verifier si on est dans un nouveau mois
        if self.Time.month == self.last_month:
            # Pas de changement -> retourner Universe.Unchanged
            return Universe.Unchanged
        
        # Nouveau mois -> mettre a jour
        self.last_month = self.Time.month
        self.Log(f"Monthly reselection: {self.Time.strftime('%Y-%m')}")
        
        # Filtrage standard
        filtered = [x for x in coarse 
                    if x.HasFundamentalData
                    and x.Price > 5 
                    and x.DollarVolume > 5000000]
        
        sorted_stocks = sorted(filtered, key=lambda x: x.DollarVolume, reverse=True)
        
        return [x.Symbol for x in sorted_stocks[:self.num_stocks]]

---

## 4. Fine Universe Selection (20 min)

### Filtrage avec donnees fondamentales

Le **Fine Universe Selection** ajoute une deuxieme couche de filtrage en utilisant les donnees fondamentales de Morningstar :

- **Valorisation** : P/E Ratio, P/B Ratio, EV/EBITDA
- **Taille** : Market Cap, Revenue, Total Assets
- **Classification** : Secteur, Industrie, Morningstar Style
- **Dividendes** : Dividend Yield, Payout Ratio

### Architecture Coarse + Fine

```python
def Initialize(self):
    # Coarse PUIS Fine
    self.AddUniverse(self.CoarseSelectionFunction, 
                     self.FineSelectionFunction)

def CoarseSelectionFunction(self, coarse):
    # Premier filtre: prix, volume
    # Retourne candidats pour Fine
    return [x.Symbol for x in filtered]

def FineSelectionFunction(self, fine):
    # Deuxieme filtre: donnees fondamentales
    # fine = seulement les actions selectionnees par Coarse
    return [x.Symbol for x in filtered]
```

### Proprietes de FineFundamental

| Categorie | Propriete | Exemple d'acces |
|-----------|-----------|------------------|
| **Valorisation** | P/E Ratio | `x.ValuationRatios.PERatio` |
| | P/B Ratio | `x.ValuationRatios.PBRatio` |
| | EV/EBITDA | `x.ValuationRatios.EVToEBITDA` |
| **Taille** | Market Cap | `x.MarketCap` |
| | Revenue | `x.FinancialStatements.IncomeStatement.TotalRevenue.Value` |
| **Classification** | Secteur Morningstar | `x.AssetClassification.MorningstarSectorCode` |
| | Industrie | `x.AssetClassification.MorningstarIndustryCode` |
| **Dividendes** | Dividend Yield | `x.ValuationRatios.TrailingDividendYield` |

In [ ]:
# Fine Universe Selection : Actions Tech avec P/E < 30
# A copier dans l'IDE QuantConnect

from AlgorithmImports import *

class TechValueUniverse(QCAlgorithm):
    """
    Selection d'actions du secteur technologie avec P/E raisonnable.
    Coarse: Volume > 10M$, Prix > 10$
    Fine: Secteur Tech, 0 < P/E < 30
    """
    
    def Initialize(self):
        self.SetStartDate(2020, 1, 1)
        self.SetEndDate(2023, 12, 31)
        self.SetCash(100000)
        
        # Universe avec Coarse ET Fine
        self.AddUniverse(self.CoarseSelectionFunction, 
                         self.FineSelectionFunction)
        
        self.num_coarse = 500  # Candidats pour Fine
        self.num_fine = 20     # Selection finale
        
        self.Log("Tech Value Universe initialized")
    
    def CoarseSelectionFunction(self, coarse):
        """Premier filtre: liquidite et HasFundamentalData."""
        # Filtrer les actions avec donnees fondamentales
        filtered = [x for x in coarse 
                    if x.HasFundamentalData  # IMPORTANT pour Fine
                    and x.Price > 10 
                    and x.DollarVolume > 10000000]
        
        # Trier par volume et prendre les top N
        sorted_stocks = sorted(filtered, key=lambda x: x.DollarVolume, reverse=True)
        
        self.Log(f"Coarse: {len(coarse)} -> {len(filtered)} filtered -> {min(len(sorted_stocks), self.num_coarse)} candidates")
        
        return [x.Symbol for x in sorted_stocks[:self.num_coarse]]
    
    def FineSelectionFunction(self, fine):
        """Deuxieme filtre: secteur et valorisation."""
        # Filtrer par secteur: Technology
        tech_stocks = [x for x in fine 
                       if x.AssetClassification.MorningstarSectorCode == MorningstarSectorCode.Technology]
        
        # Filtrer par P/E: entre 0 et 30 (exclure negatif et surevaloue)
        value_stocks = [x for x in tech_stocks 
                        if x.ValuationRatios.PERatio > 0 
                        and x.ValuationRatios.PERatio < 30]
        
        # Trier par P/E croissant (plus bas = meilleur)
        sorted_by_pe = sorted(value_stocks, 
                               key=lambda x: x.ValuationRatios.PERatio)
        
        self.Log(f"Fine: {len(fine)} -> {len(tech_stocks)} tech -> {len(value_stocks)} P/E<30 -> {min(len(sorted_by_pe), self.num_fine)} selected")
        
        return [x.Symbol for x in sorted_by_pe[:self.num_fine]]
    
    def OnSecuritiesChanged(self, changes):
        """Rebalancer quand l'univers change."""
        # Vendre les actions retirees
        for security in changes.RemovedSecurities:
            if security.Invested:
                self.Liquidate(security.Symbol)
                self.Log(f"Removed: {security.Symbol.Value}")
        
        # Acheter les nouvelles actions (equal weight)
        if len(changes.AddedSecurities) > 0:
            weight = 1.0 / max(len(self.ActiveSecurities), 1)
            for security in changes.AddedSecurities:
                self.SetHoldings(security.Symbol, weight)
                self.Log(f"Added: {security.Symbol.Value} -> {weight:.1%}")

### Codes Morningstar Sector

Les secteurs Morningstar sont accessibles via `MorningstarSectorCode` :

| Code | Secteur | Exemples |
|------|---------|----------|
| `BasicMaterials` | Materiaux de base | Mining, Chimie |
| `ConsumerCyclical` | Conso. cyclique | Retail, Automobile |
| `FinancialServices` | Services financiers | Banques, Assurances |
| `RealEstate` | Immobilier | REITs |
| `ConsumerDefensive` | Conso. defensive | Alimentation, Pharma |
| `Healthcare` | Sante | Biotech, Hopitaux |
| `Utilities` | Services publics | Electricite, Gaz |
| `CommunicationServices` | Communication | Telecom, Media |
| `Energy` | Energie | Petrole, Gaz |
| `Industrials` | Industrie | Aerospace, Transport |
| `Technology` | Technologie | Software, Hardware |

### Exemple: Selection multi-criteres

In [ ]:
# Fine Selection avec criteres multiples
# Large Cap + Dividende + Low P/E

from AlgorithmImports import *

class DividendValueUniverse(QCAlgorithm):
    """
    Selection: Large Cap + Dividend Yield > 2% + P/E < 20
    """
    
    def Initialize(self):
        self.SetStartDate(2020, 1, 1)
        self.SetEndDate(2023, 12, 31)
        self.SetCash(100000)
        
        self.AddUniverse(self.CoarseSelectionFunction, 
                         self.FineSelectionFunction)
        
        # Criteres
        self.min_market_cap = 10e9        # 10B$ minimum
        self.min_dividend_yield = 0.02    # 2% minimum
        self.max_pe_ratio = 20            # P/E max
        self.num_stocks = 30
    
    def CoarseSelectionFunction(self, coarse):
        filtered = [x for x in coarse 
                    if x.HasFundamentalData
                    and x.Price > 5 
                    and x.DollarVolume > 5000000]
        
        sorted_stocks = sorted(filtered, key=lambda x: x.DollarVolume, reverse=True)
        return [x.Symbol for x in sorted_stocks[:500]]
    
    def FineSelectionFunction(self, fine):
        # Filtre 1: Large Cap (Market Cap > 10B$)
        large_cap = [x for x in fine if x.MarketCap > self.min_market_cap]
        
        # Filtre 2: Dividend Yield > 2%
        dividend_payers = [x for x in large_cap 
                           if x.ValuationRatios.TrailingDividendYield is not None
                           and x.ValuationRatios.TrailingDividendYield > self.min_dividend_yield]
        
        # Filtre 3: P/E entre 0 et 20
        value_stocks = [x for x in dividend_payers 
                        if x.ValuationRatios.PERatio is not None
                        and 0 < x.ValuationRatios.PERatio < self.max_pe_ratio]
        
        # Trier par dividend yield (plus haut = meilleur)
        sorted_by_yield = sorted(value_stocks, 
                                  key=lambda x: x.ValuationRatios.TrailingDividendYield,
                                  reverse=True)
        
        self.Log(f"Fine: {len(fine)} total -> {len(large_cap)} large -> {len(dividend_payers)} dividend -> {len(value_stocks)} value")
        
        return [x.Symbol for x in sorted_by_yield[:self.num_stocks]]

---

## 5. Universe Settings (10 min)

### Configurer le comportement de l'univers

`UniverseSettings` permet de configurer comment QuantConnect gere les actions ajoutees dynamiquement a l'univers.

### Proprietes de UniverseSettings

| Propriete | Type | Description | Defaut |
|-----------|------|-------------|--------|
| `Resolution` | Resolution | Resolution des donnees | Minute |
| `Leverage` | float | Levier maximum | 2.0 |
| `FillForward` | bool | Remplir les donnees manquantes | True |
| `ExtendedMarketHours` | bool | Inclure pre/post-market | False |
| `MinimumTimeInUniverse` | TimeSpan | Duree minimum avant retrait | 1 jour |
| `DataNormalizationMode` | Mode | Ajustement des prix | Adjusted |

In [ ]:
# Configuration de UniverseSettings
# A copier dans l'IDE QuantConnect

from AlgorithmImports import *

class UniverseSettingsExample(QCAlgorithm):
    """
    Demonstration des UniverseSettings.
    """
    
    def Initialize(self):
        self.SetStartDate(2020, 1, 1)
        self.SetEndDate(2023, 12, 31)
        self.SetCash(100000)
        
        # === UNIVERSE SETTINGS ===
        
        # 1. Resolution des donnees pour les nouvelles securities
        # Daily = plus rapide, Minute = plus precis
        self.UniverseSettings.Resolution = Resolution.Daily
        
        # 2. Levier maximum autorise
        # 1.0 = pas de levier, 2.0 = x2 max
        self.UniverseSettings.Leverage = 1.0
        
        # 3. Fill Forward : remplir les jours sans donnees
        # avec la derniere valeur connue (utile pour actions peu liquides)
        self.UniverseSettings.FillForward = True
        
        # 4. Extended Market Hours : inclure pre-market (4am) et after-hours (8pm)
        # False = seulement 9:30am - 4:00pm
        self.UniverseSettings.ExtendedMarketHours = False
        
        # 5. Minimum Time In Universe : eviter le churn
        # Une action reste au moins 1 jour dans l'univers
        self.UniverseSettings.MinimumTimeInUniverse = timedelta(days=1)
        
        # 6. Data Normalization : ajustement des prix
        # Adjusted = ajuste pour splits/dividendes (recommande)
        self.UniverseSettings.DataNormalizationMode = DataNormalizationMode.Adjusted
        
        # Ajouter l'univers
        self.AddUniverse(self.CoarseSelectionFunction)
        
        self.Log("Universe settings configured:")
        self.Log(f"  Resolution: {self.UniverseSettings.Resolution}")
        self.Log(f"  Leverage: {self.UniverseSettings.Leverage}")
        self.Log(f"  FillForward: {self.UniverseSettings.FillForward}")
        self.Log(f"  ExtendedMarketHours: {self.UniverseSettings.ExtendedMarketHours}")
    
    def CoarseSelectionFunction(self, coarse):
        filtered = [x for x in coarse 
                    if x.HasFundamentalData
                    and x.Price > 10 
                    and x.DollarVolume > 10000000]
        
        sorted_stocks = sorted(filtered, key=lambda x: x.DollarVolume, reverse=True)
        return [x.Symbol for x in sorted_stocks[:50]]

### Quand modifier les settings?

| Scenario | Setting recommande |
|----------|--------------------|
| **Backtest rapide** | `Resolution.Daily` |
| **Strategie intraday** | `Resolution.Minute` ou `Hour` |
| **Sans levier** | `Leverage = 1.0` |
| **Actions illiquides** | `FillForward = True` |
| **Scalping pre-market** | `ExtendedMarketHours = True` |
| **Eviter overtrading** | `MinimumTimeInUniverse = timedelta(days=7)` |

---

## 6. Dynamic Rebalancing (15 min)

### Gerer les changements d'univers avec OnSecuritiesChanged

Quand l'univers change (nouvelle action ajoutee ou retiree), QuantConnect appelle `OnSecuritiesChanged(changes)`. C'est ici que vous gerez le rebalancement.

### Structure de SecurityChanges

```python
def OnSecuritiesChanged(self, changes):
    # changes.AddedSecurities : liste des nouvelles actions
    # changes.RemovedSecurities : liste des actions retirees
    
    for security in changes.RemovedSecurities:
        # Vendre la position
        pass
    
    for security in changes.AddedSecurities:
        # Acheter la nouvelle action
        pass
```

### Proprietes de Security

| Propriete | Type | Description |
|-----------|------|-------------|
| `Symbol` | Symbol | Identifiant de l'action |
| `Invested` | bool | True si position ouverte |
| `Price` | float | Prix actuel |
| `Holdings` | SecurityHolding | Details de la position |

In [ ]:
# Dynamic Rebalancing avec OnSecuritiesChanged
# A copier dans l'IDE QuantConnect

from AlgorithmImports import *

class DynamicRebalancingAlgorithm(QCAlgorithm):
    """
    Rebalancement automatique lors des changements d'univers.
    - Liquidate les actions retirees
    - Acheter les nouvelles actions en equal-weight
    """
    
    def Initialize(self):
        self.SetStartDate(2020, 1, 1)
        self.SetEndDate(2023, 12, 31)
        self.SetCash(100000)
        
        self.UniverseSettings.Resolution = Resolution.Daily
        self.AddUniverse(self.CoarseSelectionFunction)
        
        self.num_stocks = 20
        self.last_month = -1
    
    def CoarseSelectionFunction(self, coarse):
        # Reselection mensuelle
        if self.Time.month == self.last_month:
            return Universe.Unchanged
        self.last_month = self.Time.month
        
        filtered = [x for x in coarse 
                    if x.HasFundamentalData
                    and x.Price > 10 
                    and x.DollarVolume > 10000000]
        
        sorted_stocks = sorted(filtered, key=lambda x: x.DollarVolume, reverse=True)
        return [x.Symbol for x in sorted_stocks[:self.num_stocks]]
    
    def OnSecuritiesChanged(self, changes):
        """
        Appele automatiquement quand l'univers change.
        Gere le rebalancement du portfolio.
        """
        # Etape 1: Liquider les positions retirees
        for security in changes.RemovedSecurities:
            if security.Invested:
                self.Liquidate(security.Symbol)
                self.Log(f"SELL (removed): {security.Symbol.Value}")
        
        # Etape 2: Calculer le poids cible
        # Utiliser ActiveSecurities pour avoir le nombre actuel
        count = len(self.ActiveSecurities)
        if count == 0:
            return
        
        target_weight = 1.0 / count
        
        # Etape 3: Rebalancer toutes les positions
        for symbol in self.ActiveSecurities.Keys:
            self.SetHoldings(symbol, target_weight)
        
        self.Log(f"Rebalanced: {count} securities @ {target_weight:.1%} each")
    
    def OnData(self, data):
        # Pas de logique supplementaire ici
        # Le rebalancement est gere par OnSecuritiesChanged
        pass

### Patterns de rebalancement

**1. Equal-Weight** (ci-dessus) :
```python
weight = 1.0 / len(self.ActiveSecurities)
for symbol in self.ActiveSecurities.Keys:
    self.SetHoldings(symbol, weight)
```

**2. Market-Cap Weighted** :
```python
total_cap = sum(s.Fundamentals.MarketCap for s in self.ActiveSecurities.Values)
for security in self.ActiveSecurities.Values:
    weight = security.Fundamentals.MarketCap / total_cap
    self.SetHoldings(security.Symbol, weight)
```

**3. Volatility-Weighted** (Risk Parity) :
```python
# Calcul des volatilites historiques
# Allouer plus aux actions moins volatiles
```

### Eviter le over-trading

In [ ]:
# Rebalancement intelligent avec seuil de tolerance
# Evite de rebalancer pour de petites variations

from AlgorithmImports import *

class SmartRebalancingAlgorithm(QCAlgorithm):
    """
    Rebalancement avec seuil de tolerance.
    Ne rebalance que si la deviation depasse 5%.
    """
    
    def Initialize(self):
        self.SetStartDate(2020, 1, 1)
        self.SetEndDate(2023, 12, 31)
        self.SetCash(100000)
        
        self.UniverseSettings.Resolution = Resolution.Daily
        self.AddUniverse(self.CoarseSelectionFunction)
        
        self.num_stocks = 20
        self.tolerance = 0.05  # 5% de deviation max
        self.last_month = -1
    
    def CoarseSelectionFunction(self, coarse):
        if self.Time.month == self.last_month:
            return Universe.Unchanged
        self.last_month = self.Time.month
        
        filtered = [x for x in coarse 
                    if x.HasFundamentalData and x.Price > 10 and x.DollarVolume > 10000000]
        sorted_stocks = sorted(filtered, key=lambda x: x.DollarVolume, reverse=True)
        return [x.Symbol for x in sorted_stocks[:self.num_stocks]]
    
    def OnSecuritiesChanged(self, changes):
        # Liquider les retirees
        for security in changes.RemovedSecurities:
            if security.Invested:
                self.Liquidate(security.Symbol)
        
        # Rebalancer avec tolerance
        self.RebalanceWithTolerance()
    
    def RebalanceWithTolerance(self):
        """Rebalance seulement si deviation > tolerance."""
        count = len(self.ActiveSecurities)
        if count == 0:
            return
        
        target_weight = 1.0 / count
        portfolio_value = self.Portfolio.TotalPortfolioValue
        
        for symbol in self.ActiveSecurities.Keys:
            holding = self.Portfolio[symbol]
            current_weight = holding.HoldingsValue / portfolio_value if portfolio_value > 0 else 0
            
            # Rebalancer seulement si deviation > tolerance
            if abs(current_weight - target_weight) > self.tolerance:
                self.SetHoldings(symbol, target_weight)
                self.Debug(f"Rebalanced {symbol.Value}: {current_weight:.1%} -> {target_weight:.1%}")

---

## 7. Scheduled Universe Selection (10 min)

### Planifier le rebalancement periodique

Au lieu de rebalancer a chaque changement d'univers, vous pouvez planifier des rebalancements a dates fixes avec `self.Schedule.On()`.

### Schedule.On() pour rebalancement periodique

```python
# Rebalancement le premier jour du mois a 10h
self.Schedule.On(
    self.DateRules.MonthStart("SPY"),     # Date rule
    self.TimeRules.At(10, 0),              # Time rule
    self.Rebalance                         # Action
)
```

### DateRules disponibles

| DateRule | Description | Exemple |
|----------|-------------|--------|
| `EveryDay()` | Tous les jours | Rebalancement quotidien |
| `Every(day)` | Jours specifiques | `Every(DayOfWeek.Monday)` |
| `MonthStart(symbol)` | Premier jour du mois | Rebalancement mensuel |
| `MonthEnd(symbol)` | Dernier jour du mois | |
| `WeekStart(symbol)` | Premier jour de la semaine | Rebalancement hebdo |
| `WeekEnd(symbol)` | Dernier jour de la semaine | |

In [ ]:
# Scheduled Rebalancing : Mensuel
# A copier dans l'IDE QuantConnect

from AlgorithmImports import *

class ScheduledRebalancingAlgorithm(QCAlgorithm):
    """
    Rebalancement planifie le premier jour de chaque mois.
    L'univers est mis a jour mensuellement, le rebalancement aussi.
    """
    
    def Initialize(self):
        self.SetStartDate(2020, 1, 1)
        self.SetEndDate(2023, 12, 31)
        self.SetCash(100000)
        
        # Reference pour DateRules
        self.AddEquity("SPY", Resolution.Daily)
        
        self.UniverseSettings.Resolution = Resolution.Daily
        self.AddUniverse(self.CoarseSelectionFunction)
        
        self.num_stocks = 30
        self.last_month = -1
        
        # Planifier le rebalancement mensuel
        self.Schedule.On(
            self.DateRules.MonthStart("SPY"),  # Premier jour du mois
            self.TimeRules.At(10, 0),           # A 10h00
            self.Rebalance                      # Fonction a appeler
        )
        
        self.Log("Scheduled monthly rebalancing at month start, 10:00 AM")
    
    def CoarseSelectionFunction(self, coarse):
        # Mise a jour mensuelle de l'univers
        if self.Time.month == self.last_month:
            return Universe.Unchanged
        self.last_month = self.Time.month
        
        filtered = [x for x in coarse 
                    if x.HasFundamentalData and x.Price > 10 and x.DollarVolume > 10000000]
        sorted_stocks = sorted(filtered, key=lambda x: x.DollarVolume, reverse=True)
        
        self.Log(f"Universe updated: {len(sorted_stocks[:self.num_stocks])} stocks")
        return [x.Symbol for x in sorted_stocks[:self.num_stocks]]
    
    def Rebalance(self):
        """
        Fonction de rebalancement planifiee.
        Appelee automatiquement le premier jour de chaque mois.
        """
        count = len(self.ActiveSecurities)
        if count == 0:
            self.Log("Rebalance skipped: no active securities")
            return
        
        target_weight = 1.0 / count
        
        # Liquider les positions non presentes dans l'univers actuel
        for symbol in list(self.Portfolio.Keys):
            if symbol not in self.ActiveSecurities and self.Portfolio[symbol].Invested:
                self.Liquidate(symbol)
                self.Log(f"Liquidated (not in universe): {symbol.Value}")
        
        # Rebalancer vers equal-weight
        for symbol in self.ActiveSecurities.Keys:
            self.SetHoldings(symbol, target_weight)
        
        self.Log(f"Rebalanced {count} positions @ {target_weight:.1%} each")
    
    def OnSecuritiesChanged(self, changes):
        # Log les changements mais ne pas rebalancer ici
        # Le rebalancement est gere par la fonction planifiee
        if len(changes.AddedSecurities) > 0:
            self.Log(f"Added: {[s.Symbol.Value for s in changes.AddedSecurities]}")
        if len(changes.RemovedSecurities) > 0:
            self.Log(f"Removed: {[s.Symbol.Value for s in changes.RemovedSecurities]}")

### Exemples de schedules

```python
# Hebdomadaire : chaque lundi a 9h30
self.Schedule.On(
    self.DateRules.Every(DayOfWeek.Monday),
    self.TimeRules.At(9, 30),
    self.Rebalance
)

# Quotidien : a la cloture
self.Schedule.On(
    self.DateRules.EveryDay("SPY"),
    self.TimeRules.BeforeMarketClose("SPY", 30),  # 30 min avant cloture
    self.Rebalance
)

# Trimestriel : premier jour de Q1, Q2, Q3, Q4
for month in [1, 4, 7, 10]:
    self.Schedule.On(
        self.DateRules.MonthStart("SPY", month),
        self.TimeRules.At(10, 0),
        self.Rebalance
    )
```

---

## 8. Strategie Complete: Momentum Universe (20 min)

### Objectif

Construire une strategie de **momentum** complete :

- **Selection** : Top 20 actions par momentum sur 12 mois
- **Rebalancement** : Mensuel
- **Allocation** : Equal-weight

### Theorie du Momentum

Le momentum est base sur l'observation empirique que **les gagnants continuent de gagner** sur le court/moyen terme (3-12 mois). Les actions avec les meilleurs rendements passes tendent a surperformer.

**Formule du momentum** :
```
Momentum_12M = (Prix_actuel - Prix_12M_ago) / Prix_12M_ago
```

### Architecture de la strategie

```
1. Coarse Selection
   ├── Prix > 5$
   ├── Volume > 5M$/jour
   └── Retourne ~500 candidats

2. Fine Selection (Momentum)
   ├── Recupere historique 252 jours
   ├── Calcule return 12 mois
   ├── Trie par momentum decroissant
   └── Retourne top 20

3. Rebalancement mensuel
   ├── Liquide les sortants
   └── Equal-weight sur les 20
```

In [ ]:
# Strategie Momentum Universe Complete
# A copier dans l'IDE QuantConnect

from AlgorithmImports import *
from datetime import timedelta

class MomentumUniverseStrategy(QCAlgorithm):
    """
    Strategie Momentum :
    - Selection : Top 20 par momentum 12 mois
    - Rebalancement : Mensuel
    - Allocation : Equal-weight
    
    Basee sur la recherche academique de Jegadeesh & Titman (1993)
    """
    
    def Initialize(self):
        # Configuration backtest
        self.SetStartDate(2015, 1, 1)
        self.SetEndDate(2023, 12, 31)
        self.SetCash(100000)
        
        # Reference pour schedule
        self.spy = self.AddEquity("SPY", Resolution.Daily).Symbol
        
        # Universe settings
        self.UniverseSettings.Resolution = Resolution.Daily
        self.AddUniverse(self.CoarseSelectionFunction, self.FineSelectionFunction)
        
        # Parametres de la strategie
        self.num_coarse = 500         # Candidats Coarse
        self.num_fine = 20            # Selection finale
        self.lookback_days = 252      # 12 mois de trading
        self.min_price = 5            # Prix minimum
        self.min_volume = 5000000     # Volume minimum ($5M/jour)
        
        # Tracking
        self.last_month = -1
        self.momentum_scores = {}     # Stocker les scores momentum
        
        # Scheduled rebalancing
        self.Schedule.On(
            self.DateRules.MonthStart("SPY"),
            self.TimeRules.At(10, 0),
            self.Rebalance
        )
        
        self.Log("Momentum Universe Strategy initialized")
        self.Log(f"  Top {self.num_fine} stocks by {self.lookback_days}-day momentum")
        self.Log(f"  Monthly rebalancing")
    
    def CoarseSelectionFunction(self, coarse):
        """Premier filtre : liquidite."""
        # Reselection mensuelle uniquement
        if self.Time.month == self.last_month:
            return Universe.Unchanged
        self.last_month = self.Time.month
        
        # Filtrer par prix, volume et donnees fondamentales
        filtered = [x for x in coarse 
                    if x.HasFundamentalData
                    and x.Price > self.min_price 
                    and x.DollarVolume > self.min_volume]
        
        # Trier par volume
        sorted_stocks = sorted(filtered, key=lambda x: x.DollarVolume, reverse=True)
        
        self.Log(f"Coarse: {len(coarse)} total -> {len(filtered)} filtered -> {self.num_coarse} candidates")
        
        return [x.Symbol for x in sorted_stocks[:self.num_coarse]]
    
    def FineSelectionFunction(self, fine):
        """Deuxieme filtre : calcul du momentum."""
        # Calculer le momentum pour chaque action
        momentum_data = []
        
        for stock in fine:
            symbol = stock.Symbol
            
            # Recuperer l'historique 12 mois
            history = self.History(symbol, self.lookback_days, Resolution.Daily)
            
            if history.empty or len(history) < self.lookback_days * 0.9:
                # Pas assez de donnees
                continue
            
            try:
                # Calculer le momentum (return 12 mois)
                price_now = history['close'].iloc[-1]
                price_12m_ago = history['close'].iloc[0]
                
                momentum = (price_now - price_12m_ago) / price_12m_ago
                
                momentum_data.append({
                    'symbol': symbol,
                    'momentum': momentum
                })
            except:
                continue
        
        # Trier par momentum decroissant
        sorted_by_momentum = sorted(momentum_data, 
                                     key=lambda x: x['momentum'], 
                                     reverse=True)
        
        # Garder les top N
        top_momentum = sorted_by_momentum[:self.num_fine]
        
        # Stocker les scores pour reference
        self.momentum_scores = {x['symbol']: x['momentum'] for x in top_momentum}
        
        self.Log(f"Fine: {len(fine)} -> {len(momentum_data)} with data -> {len(top_momentum)} selected")
        if len(top_momentum) > 0:
            self.Log(f"  Top momentum: {top_momentum[0]['symbol'].Value} = {top_momentum[0]['momentum']:.1%}")
            self.Log(f"  Bottom of top: {top_momentum[-1]['symbol'].Value} = {top_momentum[-1]['momentum']:.1%}")
        
        return [x['symbol'] for x in top_momentum]
    
    def Rebalance(self):
        """Rebalancement mensuel en equal-weight."""
        # Exclure SPY (reference)
        trading_symbols = [s for s in self.ActiveSecurities.Keys if s != self.spy]
        
        count = len(trading_symbols)
        if count == 0:
            self.Log("Rebalance: no trading securities")
            return
        
        target_weight = 1.0 / count
        
        # Liquider les positions non presentes dans l'univers
        for symbol in list(self.Portfolio.Keys):
            if symbol not in trading_symbols and self.Portfolio[symbol].Invested:
                self.Liquidate(symbol)
                self.Log(f"Liquidated: {symbol.Value}")
        
        # Rebalancer
        for symbol in trading_symbols:
            momentum = self.momentum_scores.get(symbol, 0)
            self.SetHoldings(symbol, target_weight)
        
        self.Log(f"Rebalanced: {count} positions @ {target_weight:.1%}")
    
    def OnSecuritiesChanged(self, changes):
        """Log les changements d'univers."""
        added = [s.Symbol.Value for s in changes.AddedSecurities if s.Symbol != self.spy]
        removed = [s.Symbol.Value for s in changes.RemovedSecurities if s.Symbol != self.spy]
        
        if added:
            self.Log(f"Universe +{len(added)}: {added[:5]}{'...' if len(added) > 5 else ''}")
        if removed:
            self.Log(f"Universe -{len(removed)}: {removed[:5]}{'...' if len(removed) > 5 else ''}")
    
    def OnEndOfAlgorithm(self):
        """Resume final."""
        self.Log("="*50)
        self.Log("MOMENTUM STRATEGY SUMMARY")
        self.Log("="*50)
        self.Log(f"Initial Capital: ${self.StartingCash:,.2f}")
        self.Log(f"Final Value: ${self.Portfolio.TotalPortfolioValue:,.2f}")
        total_return = (self.Portfolio.TotalPortfolioValue - self.StartingCash) / self.StartingCash
        self.Log(f"Total Return: {total_return:.1%}")
        self.Log("="*50)

### Points cles de la strategie

1. **Coarse Selection** : Filtre rapidement ~8000 actions en ~500 candidats liquides
2. **Fine Selection** : Calcule le momentum 12 mois pour chaque candidat
3. **Rebalancement** : Equal-weight mensuel sur les top 20
4. **Scheduled** : `Schedule.On()` garantit un rebalancement previsible

### Ameliorations possibles

| Amelioration | Implementation |
|--------------|----------------|
| **Momentum ajuste au risque** | Diviser le momentum par la volatilite |
| **Exclusion du mois recent** | Skip du dernier mois (reversal) |
| **Filtrage sectoriel** | Eviter la concentration dans un secteur |
| **Stop-loss** | Vendre si perte > 20% |
| **Market timing** | Ne pas investir si SPY < SMA 200 |

---

## 9. Conclusion et Prochaines Etapes

### Recapitulatif

Dans ce notebook, nous avons appris a :

1. **Comprendre les Universes** : Selection dynamique vs liste fixe
2. **Manual Universe** : `AddEquity()` pour une liste fixe (FAANG)
3. **Coarse Selection** : Filtrage par `Price`, `DollarVolume`, `HasFundamentalData`
4. **Fine Selection** : Filtrage par donnees Morningstar (P/E, secteur, MarketCap)
5. **Universe Settings** : `Resolution`, `Leverage`, `FillForward`
6. **Dynamic Rebalancing** : `OnSecuritiesChanged()` pour gerer les changements
7. **Scheduled Selection** : `Schedule.On()` pour rebalancement periodique
8. **Strategie Momentum** : Implementation complete

### Tableau recapitulatif

| Concept | Code | Usage |
|---------|------|-------|
| Manual Universe | `self.AddEquity("AAPL")` | Liste fixe d'actions |
| Coarse Selection | `self.AddUniverse(self.CoarseSelectionFunction)` | Filtre par prix/volume |
| Fine Selection | `self.AddUniverse(Coarse, Fine)` | Filtre par fondamentaux |
| Settings | `self.UniverseSettings.Resolution` | Configurer l'univers |
| Changements | `OnSecuritiesChanged(changes)` | Gerer entrees/sorties |
| Unchanged | `return Universe.Unchanged` | Skip la reselection |
| Schedule | `self.Schedule.On()` | Rebalancement periodique |

### Workflow recommande

```
1. Definir les criteres de selection
   ├── Coarse: liquidite minimum
   └── Fine: facteurs alpha (momentum, value, quality)

2. Choisir la frequence de rebalancement
   ├── Quotidien (rare, couteux)
   ├── Hebdomadaire
   └── Mensuel (recommande)

3. Implementer OnSecuritiesChanged
   ├── Liquider les sortants
   └── Acheter les entrants

4. Backtester et optimiser
   ├── Comparer avec buy-and-hold
   └── Analyser turnover et couts
```

### Prochaine etape : QC-Py-06 - Options Trading

Dans le prochain notebook, nous explorerons :

- **Ajout d'options** : `self.AddOption("SPY")`
- **Filtrage** : Strike, expiration, type (call/put)
- **Strategies** : Covered call, protective put, spread
- **Greeks** : Delta, gamma, theta, vega

### Exercice suggere

**Creer une strategie Value Universe** :

1. Coarse: Volume > 10M$, Prix > 10$
2. Fine:
   - P/E entre 5 et 15
   - Dividend Yield > 2%
   - Market Cap > 5B$
3. Rebalancement trimestriel
4. Top 30 actions par dividend yield

### Ressources complementaires

- [Universe Selection Documentation](https://www.quantconnect.com/docs/v2/writing-algorithms/universes/equity)
- [Coarse/Fine Fundamentals](https://www.quantconnect.com/docs/v2/writing-algorithms/universes/equity/fundamental-universes)
- [Morningstar Sector Codes](https://www.quantconnect.com/docs/v2/writing-algorithms/datasets/morningstar/us-fundamental-data#05-Data-Point-Attributes)
- [Schedule Events](https://www.quantconnect.com/docs/v2/writing-algorithms/scheduled-events)

---

**Notebook complete. Pret pour QC-Py-06-Options-Trading.**